In [1]:
from geopy.distance import distance
import geopandas as gpd
import datetime
import numpy as np

file = 'clustering_example.geojson'

## Load each point in the geojson file into a pandas dataframe and create new columns for the ```coordinates``` of each point and the ```timestamp``` as a ```datetime``` object - ```timestamp_dt```

In [2]:
df = gpd.read_file(file)

def f_coords(x):
    b = x.bounds
    return (b[1], b[0])

df['coordinates'] = df['geometry'].apply(f_coords)

df["timestamp_dt"] = df["timestamp"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S"))

In [3]:
df.sort_values(by="timestamp_dt", ascending=True)

,algorithm,background_maximum_return,background_mean_return,background_minimum_return,centroid_lat,centroid_lon,change_area,compactness,detect_strength,detect_type,...,target_maximum_return,target_mean_return,target_minimum_return,target_total_return,timestamp,weighted_centroid_lat,weighted_centroid_lon,geometry,coordinates,timestamp_dt
4248,ACD-29August2019,3.36,1.27,0.04,31.592990,43.936550,526.00,1.0776,-30.9,Departure,...,2208.82,773.52,36.16,7735.24,2018-11-01T00:00:00,31.592985,43.936581,POINT Z (43.93651 31.59311 0.00000),"(31.5931135, 43.9365099)",2018-11-01
4249,ACD-29August2019,5.41,2.59,0.00,31.833372,43.656004,282.82,0.9842,-24.7,Departure,...,996.42,512.43,334.74,3074.55,2018-11-01T00:00:00,31.833377,43.656003,POINT Z (43.65597 31.83350 0.00000),"(31.8334953, 43.6559675)",2018-11-01
4250,ACD-29August2019,11.84,6.30,1.05,31.879711,43.713683,329.95,1.0846,-23.7,Departure,...,1416.18,661.18,126.76,4628.25,2018-11-01T00:00:00,31.879723,43.713676,POINT Z (43.71365 31.87983 0.00000),"(31.8798349, 43.7136467)",2018-11-01
4251,ACD-29August2019,7.58,3.16,0.29,32.835074,41.327486,479.16,1.6634,-16.2,Departure,...,259.53,151.28,51.83,1210.21,2018-11-05T00:00:00,32.835112,41.327497,POINT Z (41.32744 32.83520 0.00000),"(32.8351977, 41.3274391)",2018-11-05
4252,ACD-29August2019,12.07,4.11,0.00,32.502141,41.416089,359.23,1.0462,-27.2,Departure,...,303.04,161.45,60.37,968.68,2018-11-05T00:00:00,32.502158,41.416095,POINT Z (41.41604 32.50226 0.00000),"(32.5022643, 41.4160425)",2018-11-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4218,ACD-29August2019,1.82,0.71,0.23,31.007855,42.221301,422.92,0.9998,40.0,Arrival,...,12574.07,3746.76,37.01,33720.83,2019-10-30T00:00:00,31.007813,42.221297,POINT Z (42.22127 31.00798 0.00000),"(31.0079784, 42.2212653)",2019-10-30
4219,ACD-29August2019,2.28,1.59,0.45,31.022819,42.367975,281.94,1.0310,29.0,Arrival,...,502.72,295.57,48.38,1773.42,2019-10-30T00:00:00,31.022863,42.367972,POINT Z (42.36794 31.02294 0.00000),"(31.0229429, 42.367939)",2019-10-30
4220,ACD-29August2019,6.27,1.34,0.28,31.075552,42.526360,328.93,1.0716,27.2,Arrival,...,904.40,385.43,38.03,2698.03,2019-10-30T00:00:00,31.075552,42.526361,POINT Z (42.52632 31.07568 0.00000),"(31.0756762, 42.5263246)",2019-10-30
4222,ACD-29August2019,14.63,7.20,0.33,30.996824,42.574917,328.93,1.0846,23.2,Arrival,...,1329.47,494.65,105.10,3462.54,2019-10-30T00:00:00,30.996845,42.574910,POINT Z (42.57488 30.99695 0.00000),"(30.9969476, 42.5748815)",2019-10-30


## ```dts``` in the following cell will be a list of unique ```datetime``` objects from geojson file##

In [4]:
list_dts = df["timestamp_dt"].to_list()
np_dts = np.array(list_dts)
dts = np.unique(np_dts)

In [5]:
class Cluster:
    def __init__(self, coords, time, size):
        self.coords = coords
        self.time = time
        self.size = size
    def new_point(self, coord):
        self.coords.append(coord)
        self.size += 1
    def print_cluster(self):
        print("Date of cluster: ", end = " ")
        print(self.time)
        print("Size of cluster: ", end = " ")
        print(self.size)
        print("Point within collect: ")
        for coord in self.coords:
            print(coord.get_coords())
    def get_coords(self):
        return self.coords
    def add_coords(self, coords):
        self.coords.extend(coords)
        self.size += len(coords)
    def get_size(self):
        return self.size
    
    # functions used to make circles
    def make_centroid(self):
        tot_lat = 0
        tot_lon = 0
        for coord in self.coords:
            c = coord.get_coords()
            tot_lat += c[0]
            tot_lon += c[1]
        lat = tot_lat / len(self.coords)
        lon = tot_lon / len(self.coords)
        self.centroid = (lat, lon)
    def get_centroid(self):
        return self.centroid
    def make_radii(self):
        dist_sum = 0
        dist_max = 0
        for coord in self.coords:
            c = coord.get_coords()
            dist = distance(c, self.centroid).km
            if dist > dist_max:
                dist_max = dist
            dist_sum += dist
        self.max_r = dist_max
        self.avg_r = dist_sum / len(self.coords)
    def get_max_r(self):
        return self.max_r
    def get_avg_r(self):
        return self.avg_r

In [6]:
class Collect(Cluster):
    def __init__(self, coords, time, num, clusters):
        self.coords = coords
        self.time = time
        self.num = num
        self.clusters = clusters
    def print_collect(self):
        print("Date of collect: ", end = " ") 
        print(self.time)
        print("Number of points in collect: ", end = " ") 
        print(self.num)
        print("Coordinates of points: ", end = " ") 
        print(self.coords)
    def get_coords(self):
        return self.coords
    def get_time(self):
        return self.time
    def get_clusters(self):
        return self.clusters
    def get_cluster(self, i):
        return self.clusters[i]
    def combine_clusters(self, i1, i2):
        c_list = self.clusters[i2].get_coords()
        self.clusters[i1].add_coords(c_list)
        dup_clust = self.clusters[i2]
        self.clusters.remove(dup_clust)
    def add_to_cluster(self, i, cc):
        self.clusters[i].new_point(cc)
    def add_cluster(self, cc):
        self.clusters.append(Cluster(cc, self.time, 2))
    def print_clusters(self):
        idx = 0
        for cluster in self.clusters:
            print("Index in clusters array: " + str(idx))
            self.clusters[idx].print_cluster()
            idx += 1
            print()
    def print_cluster_distances(self):
        idx = 0
        for cluster in self.clusters:
            print("INDEX OF CLUSTER IN ARRAY: " + str(idx))
            coords = cluster.get_coords()
            for i in range(len(coords)):
                for j in range(len(coords)):
                    if i < j:
                        print("Points " + str(i) + " and " + str(j))
                        print("First point: " + str(coords[i].get_coords()))
                        print("Second point: " + str(coords[j].get_coords()))
                        dist = distance(coords[i].get_coords(), coords[j].get_coords()).km
                        print("Distance in km between the points: " + str(dist))
                        print()
            idx += 1
    def make_circle_properties(self):
        for cluster in self.clusters:
            cluster.make_centroid()
            cluster.make_radii()
            print(cluster.get_centroid())
            print(cluster.get_max_r())
            print(cluster.get_avg_r())
            print()
    def wipe_clusters(self):
        self.clusters = []
        
#     def debug(self):
#         clust1 = self.get_cluster(2)
#         clust2 = self.get_cluster(3)
#         coords1 = clust1.get_coords()
#         coords2 = clust2.get_coords()
#         for i in range(len(coords1)):
#                 for j in range(len(coords2)):
#                     print("Points " + str(i) + " and " + str(j))
#                     print("First point: " + str(coords1[i].get_coords()))
#                     print("Second point: " + str(coords2[j].get_coords()))
#                     dist = distance(coords1[i].get_coords(), coords2[j].get_coords()).km
#                     print("Distance in km between the points: " + str(dist))
#                     print()

In [7]:
# built to keep track of whether a set of coordinates is already in a cluster with c

class Cluster_Coord:
    def __init__(self, coords, c):
        self.coords = coords
        self.c = c
    def change_status(self, c):
        self.c = c
    def get_status(self):
        return self.c
    def get_coords(self):
        return self.coords

In [8]:
collects = [] # list of collects

In [9]:
def init_collect(time):
    coords = []
    collect_data = df.loc[df['timestamp_dt']==time, ['coordinates', 'timestamp_dt']]
    num = len(collect_data)
    for i in range(num):
        d = collect_data.iloc[i,:]
        coords.append(d[0])
    clusters = []
    collects.append(Collect(coords, time, num, clusters))

In [10]:
# initialize all of the collects to the collects array - can be different time period down the line

for time in dts:
    init_collect(time)

idx = 0
for collect in collects:
    print("Index of collects array: " + str(idx))
    idx += 1
    collect.print_collect()
    print()

Index of collects array: 0
Date of collect:  2018-11-01 00:00:00
Number of points in collect:  3
Coordinates of points:  [(31.5931135, 43.9365099), (31.8334953, 43.6559675), (31.8798349, 43.7136467)]

Index of collects array: 1
Date of collect:  2018-11-05 00:00:00
Number of points in collect:  4
Coordinates of points:  [(32.8351977, 41.3274391), (32.5022643, 41.4160425), (32.4978841, 41.4141477), (32.4518448, 41.37964)]

Index of collects array: 2
Date of collect:  2018-11-06 00:00:00
Number of points in collect:  23
Coordinates of points:  [(32.7993297, 42.8858036), (32.3368143, 43.319328), (32.3849988, 43.3193711), (32.384144, 43.3173443), (32.3437482, 43.3284922), (32.343872, 43.3279943), (32.3373096, 43.3204349), (32.3439545, 43.3276754), (31.5390979, 43.4076214), (32.431786, 42.1213852), (32.4875081, 42.1089776), (32.2772893, 42.209378), (32.0276532, 42.3203179), (32.0953926, 42.5449025), (32.0901751, 42.5058812), (32.0380529, 42.2591599), (31.5625085, 42.1659556), (31.5403554, 4

## The following two cells are to show the distances between each point in a collect. This distance is used to form clusters

In [11]:
def print_collect_distances(collect):
    coords = collect.get_coords()
    for i in range(len(coords)):
        for j in range(len(coords)):
            if i < j:
                print("Points " + str(i) + " and " + str(j))
                print("First point: " + str(coords[i]))
                print("Second point: " + str(coords[j]))
                dist = distance(coords[i], coords[j]).km
                print("Distance in km between the points: " + str(dist))
                print()

In [12]:
print_collect_distances(collects[17])

Points 0 and 1
First point: (33.0813515, 42.5993827)
Second point: (32.6721385, 42.9268105)
Distance in km between the points: 54.75843494591403

Points 0 and 2
First point: (33.0813515, 42.5993827)
Second point: (32.357043, 43.3199236)
Distance in km between the points: 104.95207106055506

Points 0 and 3
First point: (33.0813515, 42.5993827)
Second point: (32.3849898, 43.3194997)
Distance in km between the points: 102.566505692264

Points 0 and 4
First point: (33.0813515, 42.5993827)
Second point: (32.3436552, 43.3251394)
Distance in km between the points: 106.40783854958119

Points 0 and 5
First point: (33.0813515, 42.5993827)
Second point: (32.3427997, 43.3188364)
Distance in km between the points: 106.10436377174963

Points 0 and 6
First point: (33.0813515, 42.5993827)
Second point: (31.7296639, 43.2709788)
Distance in km between the points: 162.66388046924877

Points 0 and 7
First point: (33.0813515, 42.5993827)
Second point: (31.5389336, 43.0494765)
Distance in km between the poi

Distance in km between the points: 235.4226483137747

Points 1 and 74
First point: (32.6721385, 42.9268105)
Second point: (31.0614991, 44.5466498)
Distance in km between the points: 235.34768778502394

Points 1 and 75
First point: (32.6721385, 42.9268105)
Second point: (31.0593728, 44.5477394)
Distance in km between the points: 235.59486677153055

Points 1 and 76
First point: (32.6721385, 42.9268105)
Second point: (31.0601985, 44.5477574)
Distance in km between the points: 235.5260566152703

Points 1 and 77
First point: (32.6721385, 42.9268105)
Second point: (31.0600048, 44.5484526)
Distance in km between the points: 235.5852970321943

Points 1 and 78
First point: (32.6721385, 42.9268105)
Second point: (31.0794927, 44.6085425)
Distance in km between the points: 237.70696328133545

Points 1 and 79
First point: (32.6721385, 42.9268105)
Second point: (31.0838775, 44.6227473)
Distance in km between the points: 238.24653072184736

Points 1 and 80
First point: (32.6721385, 42.9268105)
Second

Distance in km between the points: 92.95645897712032

Points 4 and 11
First point: (32.3436552, 43.3251394)
Second point: (31.538468, 43.0476208)
Distance in km between the points: 93.05982842352444

Points 4 and 12
First point: (32.3436552, 43.3251394)
Second point: (31.465778, 43.1575344)
Distance in km between the points: 98.62606700909349

Points 4 and 13
First point: (32.3436552, 43.3251394)
Second point: (31.4663766, 43.1603434)
Distance in km between the points: 98.5181625049297

Points 4 and 14
First point: (32.3436552, 43.3251394)
Second point: (31.5507179, 43.1509238)
Distance in km between the points: 89.4550654380217

Points 4 and 15
First point: (32.3436552, 43.3251394)
Second point: (31.4835504, 42.9939834)
Distance in km between the points: 100.38420296435238

Points 4 and 16
First point: (32.3436552, 43.3251394)
Second point: (31.4454822, 43.1387586)
Distance in km between the points: 101.14252759424677

Points 4 and 17
First point: (32.3436552, 43.3251394)
Second point

First point: (32.3427997, 43.3188364)
Second point: (31.7297252, 43.2709806)
Distance in km between the points: 68.1323787405202

Points 5 and 89
First point: (32.3427997, 43.3188364)
Second point: (31.5550203, 43.1030081)
Distance in km between the points: 89.70525888007536

Points 5 and 90
First point: (32.3427997, 43.3188364)
Second point: (31.4422468, 43.1064638)
Distance in km between the points: 101.85875768982147

Points 5 and 91
First point: (32.3427997, 43.3188364)
Second point: (31.3205054, 43.3479498)
Distance in km between the points: 113.38945640907237

Points 5 and 92
First point: (32.3427997, 43.3188364)
Second point: (31.206765, 43.3454302)
Distance in km between the points: 125.9919967971973

Points 5 and 93
First point: (32.3427997, 43.3188364)
Second point: (32.2795994, 42.2083245)
Distance in km between the points: 104.81387593103186

Points 5 and 94
First point: (32.3427997, 43.3188364)
Second point: (32.0241244, 42.4982753)
Distance in km between the points: 85.06

Points 8 and 37
First point: (31.5391334, 43.0483038)
Second point: (32.2007478, 41.0457492)
Distance in km between the points: 203.19525151142923

Points 8 and 38
First point: (31.5391334, 43.0483038)
Second point: (30.6157908, 44.5250406)
Distance in km between the points: 174.17627984375181

Points 8 and 39
First point: (31.5391334, 43.0483038)
Second point: (30.8245379, 44.6062714)
Distance in km between the points: 168.31916097287407

Points 8 and 40
First point: (31.5391334, 43.0483038)
Second point: (30.8245224, 44.6068154)
Distance in km between the points: 168.36573178645236

Points 8 and 41
First point: (31.5391334, 43.0483038)
Second point: (30.730026, 44.6899771)
Distance in km between the points: 180.44125021163643

Points 8 and 42
First point: (31.5391334, 43.0483038)
Second point: (30.6724639, 44.6471329)
Distance in km between the points: 180.2663234191333

Points 8 and 43
First point: (31.5391334, 43.0483038)
Second point: (30.825891, 44.6059349)
Distance in km between

...

Second point: (31.206765, 43.3454302)
Distance in km between the points: 148.73614339786687

Points 62 and 93
First point: (30.7445994, 44.8071214)
Second point: (32.2795994, 42.2083245)
Distance in km between the points: 299.8277195922368

Points 62 and 94
First point: (30.7445994, 44.8071214)
Second point: (32.0241244, 42.4982753)
Distance in km between the points: 261.44137442979684

Points 62 and 95
First point: (30.7445994, 44.8071214)
Second point: (31.8081367, 42.4893512)
Distance in km between the points: 250.2312229529025

Points 62 and 96
First point: (30.7445994, 44.8071214)
Second point: (31.5629359, 42.1658336)
Distance in km between the points: 267.6835666192218

Points 62 and 97
First point: (30.7445994, 44.8071214)
Second point: (31.5588498, 42.0699299)
Distance in km between the points: 276.16023068850717

Points 62 and 98
First point: (30.7445994, 44.8071214)
Second point: (31.4406201, 41.7033686)
Distance in km between the points: 306.0111294683044

Points 62 and 99


Points 68 and 100
First point: (30.9411214, 44.7290764)
Second point: (32.7771693, 41.2546692)
Distance in km between the points: 386.6967280094285

Points 68 and 101
First point: (30.9411214, 44.7290764)
Second point: (32.7771791, 41.2546577)
Distance in km between the points: 386.6982102140298

Points 68 and 102
First point: (30.9411214, 44.7290764)
Second point: (32.73803, 41.1417127)
Distance in km between the points: 393.67114961612657

Points 68 and 103
First point: (30.9411214, 44.7290764)
Second point: (32.4152807, 41.5928679)
Distance in km between the points: 339.31799292904964

Points 68 and 104
First point: (30.9411214, 44.7290764)
Second point: (31.9064806, 41.6184043)
Distance in km between the points: 314.5166468099276

Points 69 and 70
First point: (30.8257908, 44.6062599)
Second point: (30.960827, 44.4203549)
Distance in km between the points: 23.239321670571428

Points 69 and 71
First point: (30.8257908, 44.6062599)
Second point: (31.114555, 44.5837117)
Distance in km

Points 73 and 98
First point: (31.0605944, 44.5466231)
Second point: (31.4406201, 41.7033686)
Distance in km between the points: 274.0797814828447

Points 73 and 99
First point: (31.0605944, 44.5466231)
Second point: (31.5394475, 41.9382217)
Distance in km between the points: 253.93511451866604

Points 73 and 100
First point: (31.0605944, 44.5466231)
Second point: (32.7771693, 41.2546692)
Distance in km between the points: 364.8840108781615

Points 73 and 101
First point: (31.0605944, 44.5466231)
Second point: (32.7771791, 41.2546577)
Distance in km between the points: 364.88549122622794

Points 73 and 102
First point: (31.0605944, 44.5466231)
Second point: (32.73803, 41.1417127)
Distance in km between the points: 371.9079506208023

Points 73 and 103
First point: (31.0605944, 44.5466231)
Second point: (32.4152807, 41.5928679)
Distance in km between the points: 317.63960777452456

Points 73 and 104
First point: (31.0605944, 44.5466231)
Second point: (31.9064806, 41.6184043)
Distance in 

Points 82 and 97
First point: (32.3454189, 43.3252521)
Second point: (31.5588498, 42.0699299)
Distance in km between the points: 147.28123516101897

Points 82 and 98
First point: (32.3454189, 43.3252521)
Second point: (31.4406201, 41.7033686)
Distance in km between the points: 183.32071707805153

Points 82 and 99
First point: (32.3454189, 43.3252521)
Second point: (31.5394475, 41.9382217)
Distance in km between the points: 158.7001614345926

Points 82 and 100
First point: (32.3454189, 43.3252521)
Second point: (32.7771693, 41.2546692)
Distance in km between the points: 200.258475764706

Points 82 and 101
First point: (32.3454189, 43.3252521)
Second point: (32.7771791, 41.2546577)
Distance in km between the points: 200.25977392768064

Points 82 and 102
First point: (32.3454189, 43.3252521)
Second point: (32.73803, 41.1417127)
Distance in km between the points: 209.6729911565929

Points 82 and 103
First point: (32.3454189, 43.3252521)
Second point: (32.4152807, 41.5928679)
Distance in km

Points 93 and 104
First point: (32.2795994, 42.2083245)
Second point: (31.9064806, 41.6184043)
Distance in km between the points: 69.37485317829048

Points 94 and 95
First point: (32.0241244, 42.4982753)
Second point: (31.8081367, 42.4893512)
Distance in km between the points: 23.964738180408137

Points 94 and 96
First point: (32.0241244, 42.4982753)
Second point: (31.5629359, 42.1658336)
Distance in km between the points: 60.05255264917814

Points 94 and 97
First point: (32.0241244, 42.4982753)
Second point: (31.5588498, 42.0699299)
Distance in km between the points: 65.63007357566188

Points 94 and 98
First point: (32.0241244, 42.4982753)
Second point: (31.4406201, 41.7033686)
Distance in km between the points: 99.30097684079243

Points 94 and 99
First point: (32.0241244, 42.4982753)
Second point: (31.5394475, 41.9382217)
Distance in km between the points: 75.51227209557666

Points 94 and 100
First point: (32.0241244, 42.4982753)
Second point: (32.7771693, 41.2546692)
Distance in km 

In [13]:
def find_cluster(collect, cc): #pass in 2 cluster_coords
    for i in range(len(collect.get_clusters())):
        for point in collect.get_cluster(i).get_coords():
            if point == cc:
                return i
    return -1

In [14]:
def form_clusters_by_collect(collect, km_dist):
    cc = []
    coords = collect.get_coords()
    #init Cluster_Coords
    for coord in coords:
        cc.append(Cluster_Coord(coord, False))
    for i in range(len(cc)):
        for j in range(len(cc)):
            if (j > i):
                c1 = cc[i].get_coords()
                c2 = cc[j].get_coords()
                dist = distance(c1, c2).km
                if dist <= km_dist:
                    if cc[i].get_status() == True and cc[j].get_status() == True:
                        # check if in same cluster
                        clust1 = find_cluster(collect, cc[i])
                        clust2 = find_cluster(collect, cc[j])
                        if clust1 != clust2:
                            # concatenate clusters
                            collect.combine_clusters(clust1, clust2)

                    elif cc[i].get_status() == True and cc[j].get_status() == False:
                        clust = find_cluster(collect, cc[i])
                        collect.add_to_cluster(clust, cc[j])
                        cc[j].change_status(True)
                        
                    elif cc[i].get_status() == False and cc[j].get_status() == True:
                        clust = find_cluster(collect, cc[j])
                        collect.add_to_cluster(clust, cc[i])
                        cc[i].change_status(True)
                        
                    else: 
                        # create new cluster
                        collect.add_cluster([cc[i], cc[j]])
                        cc[i].change_status(True)
                        cc[j].change_status(True)

In [15]:
for collect in collects:
    form_clusters_by_collect(collect, 10)

## As I did with collects, I now list the distances between each point in each cluster (at a certain collect). Hopefully, this can help us make informed decisions about the effectiveness of a chosen ```km_dist``` threshold

## I now make new cluster properties in order to create visual representations of clusters

In [16]:
for collect in collects:
    collect.make_circle_properties()


(31.8566651, 43.6848071)
3.748592777210732
3.7483474932968717

(32.48399773333333, 41.403276733333335)
4.2013329766569285
2.80123775208644

(32.35354877142857, 43.3229486)
3.5038111576424713
2.0246815401113003

(32.45964705, 42.1151814)
3.1442319241660375
3.144221996880522

(32.03285305, 42.2897389)
2.9455430641107334
2.945463077908622

(32.092783850000004, 42.52539185)
1.8643692363790463
1.8643433945358745

(31.5402117, 41.9634825)
4.542038077732417
3.028022155564335

(31.327782900000003, 41.441635000000005)
1.2394357248301042
1.239435223090609

(31.504152400000002, 42.578779600000004)
4.8454296150745435
4.845022233133797

(31.520130799999997, 42.0088762)
4.3768704722875595
4.376816566058645

(31.61068075, 42.235844)
8.548683437251356
5.049180161955631

(31.9650643, 42.51505885)
2.131573878384552
2.1315084066649304

(32.183544749999996, 41.955313849999996)
0.6291816096915077
0.6291786583360452

(31.422977799999998, 41.352782033333334)
0.05647660083859239
0.03347023492825623

(31.53238

2.4938837606238633

(31.66309475, 42.319180349999996)
2.659396490955523
2.6593809615053186

(31.498977099999998, 41.96213665)
2.053633092689262
2.0535574474715377

(31.23313435, 42.06958195)
1.8948805719195594
1.894877970876049

(31.371706183333334, 41.45166703333333)
6.127757340427852
2.957164740258108

(31.514786333333333, 41.54926483333333)
3.690241480933379
2.5004368559210715

(32.21541535, 40.50045)
2.2796758183191352
2.2796680106391927

(30.712802971428573, 42.15635534285714)
7.507695124534542
5.79710328351674

(30.874349066666667, 41.92310043333333)
11.119398048939338
7.445970501510371

(29.90501152857143, 43.05615785714285)
6.070482392403469
3.8909256201445404

(30.008054657142853, 42.83571352857143)
2.7405374591913896
1.4975534885751944

(30.08766275, 43.0473391)
0.0018844624309987963
0.0018844624063047827

(30.109693200000002, 42.6932378)
0.0013130894629126797
0.0013130894585253689

(30.388816030769235, 42.55486145384615)
9.358312208130547
4.335609887067772

(30.2864031466666

(30.02146226428571, 42.804447185714295)
15.392342434471503
7.294562969428705

(29.9846826, 43.17268665)
0.0019256615773645535
0.0019256615472086971

(30.05770505, 43.0994328)
0.002921885343224963
0.0029218853428747017

(30.184158479999997, 42.721448020000004)
4.5428742617783
3.9081736435879817

(30.372984, 42.7674499)
0.0018562384991017262
0.0018562384763360995

(30.52145185, 42.4993914)
3.736311528165535
3.736163639992978

(31.25624145, 43.1130955)
0.0020053225405606096
0.0020053224945518477

(30.366882425, 43.76892745)
4.824622665001031
3.5104565970078347

(30.44244431666667, 43.482677216666666)
11.328004823003148
8.69163137088107

(30.5878789, 43.720205)
0.24343617942811527
0.24343531838194887

(30.6243844, 43.35862337142857)
9.589639828337102
6.870985352760047

(30.73479185, 43.9151424)
0.0028957060211368913
0.002895706017607518

(30.9047269, 43.86940345)
0.005183721821452471
0.00518372180637976

(31.0245547, 43.290304500000005)
0.005235288037899223
0.005235288019856416

(31.062503

(31.97987175, 42.584721599999995)
4.800787721101667
4.800361811064969

(32.039128166666664, 42.25393626666666)
0.43556909190733895
0.32453538614640265

(32.356624249999996, 43.3192411)
0.0045957173853590095
0.004595717336730307

(31.500903233333332, 43.285054566666666)
4.8872795505378175
3.25836873513327

(31.30071315, 43.148407649999996)
0.0026975797138770134
0.002697579703002919

(32.45371293333333, 42.08669643333334)
3.0563567200447967
2.0375697283939256

(32.129242233333336, 42.5317894)
5.5044674721203695
3.6698332496088106

(32.0673286, 42.615728149999995)
4.721201695202278e-06
4.721201359740748e-06

(31.7850647, 42.389166875)
3.66353092044238
3.6628864636514686

(31.441890450000002, 42.5201366)
0.0025452683688811885
0.002545268365239809

(31.45265955, 41.80714565)
8.098261544831658e-05
8.098261535437396e-05

(29.9243077, 43.00823805)
3.4698597295979745
3.469657525320498

(30.8438619, 42.019794475)
8.47367513453142
5.839575713252252

(30.561965400000002, 42.463455893749995)
21.745

(30.2601046, 42.61504725)
1.0150286478743884
1.015024248629414

(30.55564735, 42.4864447)
0.009883815149542395
0.00988381512318492

(30.7096909, 42.20303065)
2.136315516635176
2.136272625171243

(30.939790600000002, 42.7835152)
4.7529566105529195
4.7528448222714115

(31.166108484615386, 42.47517878461539)
19.11015309459461
10.518962148737081

(31.392515833333334, 42.353146233333334)
3.558596408528976
2.3722853681850258

(32.7143415, 41.1504975)
0.14584786038851635
0.1458478436257481

(31.653637011111112, 42.31864937777778)
24.249491527653145
10.521784162177251

(31.90089555, 42.4942635)
4.193459210250372
4.193149390686862

(32.087305725, 42.51509345)
7.472734238867
4.748793056005473

(32.04019056666667, 42.2792629)
5.033499619602385
3.368310470080027

(32.228831799999995, 42.249652850000004)
0.013985174874667178
0.013985174800698705

(32.40222815, 42.23413025)
0.0010689541428913233
0.0010689541397693235

(32.519951433333325, 42.217391899999996)
4.687510649582548
3.147719588574971

(33.

(31.345728488888888, 42.343043533333336)
6.85091979340951
4.180254598530173

(31.275065100000003, 41.432473433333335)
6.11864301524383
4.079114777114724

(30.4452785, 42.37828835)
4.194552627250967
4.194256303647567

(30.64265251111111, 42.26815499999999)
11.411909317997663
7.384577962894365

(31.0540003, 42.503531699999996)
0.0250336707300617
0.02503365944333693

(31.04245345, 42.2787566)
4.728440476862662
4.72834876757522

(31.0564993, 42.70928955)
4.824725900315771
4.824382071423026

(31.303675600000002, 42.14476065)
9.58338541318949
6.831310918302111

(31.4338974, 42.6498708)
0.0003259494166763138
0.0003259494165227582

(31.4278693, 42.372415425)
10.0563910772862
6.190434830144094

(32.3657618, 43.3194394)
2.1668683559506716
1.445622781201644

(32.95084533333333, 42.36590833333333)
1.2151895267757409
0.8101419732742791

(32.680125825, 42.316739775)
0.0034334780228087534
0.0024881416708094728

(32.347829600000004, 43.32025816666667)
0.868549339126988
0.5790425356060296

(31.9875074,

## Finally, using Leaflet and the ```folium``` module, I graph the clusters as red circles

In [27]:
import folium
import webbrowser
from folium_jsbutton import JsButton
import os
import matplotlib

In [28]:
# find the best center location for the Leaflet map

coordinates = df["coordinates"].to_list()
lat_sum = 0
lon_sum = 0
for coord in coordinates:
    lat_sum += coord[0]
    lon_sum += coord[1]
leaf_lat = lat_sum / len(coordinates)
leaf_lon = lon_sum / len(coordinates)

In [53]:
def make_maps():
    os.chdir("map_outputs")
    file_cnt = 1
    for collect in collects:
        m = folium.Map(location=[leaf_lat, leaf_lon], zoom_start=8)
        clust_cnt = 1
        for cluster in collect.get_clusters():
            r = cluster.get_avg_r() * 1000
            c = cluster.get_centroid()
            s = cluster.get_size()
            if (s*30) > 255:
                red = 1
            else:
                red = (s*30)/255
            blue = 1 - red
            rgb = (red, 0, blue)
            hfc = matplotlib.colors.to_hex([red, 0, blue])
            folium.Circle(
                location = c,
                radius = r,
                color = 'gray',
                fill = True,
                fill_color = hfc,
                popup = "#" + str(clust_cnt) + "\n Size: " + str(s)
            ).add_to(m)
            clust_cnt += 1
        JsButton(
            title='<i class="fas fa-backward"></i>',function="""
            function(btn, map) {
                location.replace("file:///home/conor/Projects/D4D/spotthedifference/conor/map_outputs/map"""
                + str(file_cnt-1) +
                """.html"); 
            }
            """).add_to(m)
        JsButton(
            title='<i class="fas fa-forward"></i>',function="""
            function(btn, map) {
                location.replace("file:///home/conor/Projects/D4D/spotthedifference/conor/map_outputs/map"""
                + str(file_cnt+1) +
                """.html"); 
            }
            """).add_to(m)
        m.save('map' + str(file_cnt) + '.html')
        file_cnt += 1
    
    os.chdir("..")

In [54]:
make_maps()

In [55]:
# Change for local machine!!
url = "file:///home/conor/Projects/D4D/spotthedifference/conor/map_outputs/map1.html"

webbrowser.open(url, new=2) # open in new tab

True